In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def load_data():
    return pd.read_csv('medical_examination.csv')

def add_overweight_column(df):
    # Calculate BMI and determine overweight
    df['overweight'] = (df['weight'] / ((df['height'] / 100) ** 2) > 25).astype(int)
    return df

def normalize_data(df):
    # Normalize cholesterol and gluc columns
    df['cholesterol'] = (df['cholesterol'] > 1).astype(int)
    df['gluc'] = (df['gluc'] > 1).astype(int)
    return df

def draw_cat_plot(df):
    # Create DataFrame for cat plot using pd.melt
    df_cat = pd.melt(df, id_vars=['cardio'], value_vars=['cholesterol', 'gluc', 'smoke', 'alco', 'active', 'overweight'])

    # Group and reformat the data for the catplot
    df_cat = df_cat.groupby(['cardio', 'variable', 'value']).size().reset_index(name='total')

    # Draw the catplot
    catplot = sns.catplot(
        data=df_cat,
        x='variable', y='total', hue='value', col='cardio',
        kind='bar', height=5, aspect=1
    )

    fig = catplot.fig
    return fig

def draw_heat_map(df):
    # Clean the data
    df_heat = df[(df['ap_lo'] <= df['ap_hi']) &
                 (df['height'] >= df['height'].quantile(0.025)) &
                 (df['height'] <= df['height'].quantile(0.975)) &
                 (df['weight'] >= df['weight'].quantile(0.025)) &
                 (df['weight'] <= df['weight'].quantile(0.975))]

    # Calculate the correlation matrix
    corr = df_heat.corr()

    # Generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=bool))

    # Set up the matplotlib figure
    fig, ax = plt.subplots(figsize=(10, 8))

    # Draw the heatmap
    sns.heatmap(
        corr,
        mask=mask,
        annot=True,
        fmt='.1f',
        cmap='coolwarm',
        square=True,
        cbar_kws={'shrink': .5},
        ax=ax
    )

    return fig

if __name__ == "__main__":
    # Load data
    df = load_data()

    # Process data
    df = add_overweight_column(df)
    df = normalize_data(df)

    # Draw and save plots
    cat_plot = draw_cat_plot(df)
    cat_plot.savefig('catplot.png')

    heat_map = draw_heat_map(df)
    heat_map.savefig('heatmap.png')
